In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import KFold
pd.set_option('display.max_columns',50)
import re
import timeit
import random
random.seed(3)

In [4]:
import xgboost

In [5]:
import nltk

In [6]:
start=timeit.default_timer()
train=pd.read_csv('C:\\Users\pragy\OneDrive\Documents\ml-practice datasets\ml1\\train_indessa.csv')
test=pd.read_csv('C:\\Users\pragy\OneDrive\Documents\ml-practice datasets\ml1\\test_indessa.csv')
stop=timeit.default_timer()
print(- start + stop)

6.337591177816754


In [7]:
target=train.loan_status.values
train.drop('loan_status',axis=1,inplace=True)

In [8]:
size_train=train.shape[0]
size_test=test.shape[0]
full_df=train.append(test)
n=full_df.isnull().sum(axis=1)
full_df['nulls']=n


In [9]:
drop_cols=['emp_title','mths_since_last_major_derog','batch_enrolled','member_id','mths_since_last_record']
full_df.drop(drop_cols,axis=1,inplace=True)

In [10]:
def clean_title(raw):
    if('debt' in raw) or ('consol' in raw) or ('lidation' in raw):
        return 'debt consolidation'
    elif('credit' in raw) or ('card' in raw):
        return 'credit card'
    elif ('home'in raw) or ('house' in raw) or ('bath' in raw):
        return 'home'
    elif ('car' in raw) or ('motor' in raw) or ('truck' in raw) or ('auto' in raw):
        return 'vehicle'
    elif ('payoff' in raw) or ('off' in raw):
        return 'payoff'
    elif ('wedd' in raw) or ('marr' in raw):
        return 'wedding'
    elif('freedom' in raw):
        return 'freedom'
    elif('pool' in raw):
        return 'pool'
    elif('persona' in raw):
        return 'personal'
    elif('vacation' in raw):
        return 'vacation'
    elif('hosp' in raw) or ('med' in raw):
        return 'medical'
    elif('loan' in raw):
        return 'some loan'
    elif('movin' in raw):
        return 'moving'
    else:
        return raw
    
full_df['title']=full_df.title.apply(lambda x:x.lower() if type(x)==str else '')
full_df['title']=full_df.title.apply(clean_title)

In [11]:
title_list=full_df.title.values
title_list

array(['debt consolidation', 'home', 'debt consolidation', ...,
       'debt consolidation', 'credit card', 'debt consolidation'],
      dtype=object)

In [12]:
from collections import Counter
wordsDict=Counter(title_list)
wordsDictSorted=wordsDict.most_common()
wordsDictSorted=dict(wordsDictSorted)
title_list=[]
for i in wordsDictSorted:
    if(wordsDictSorted[i]>1):
        title_list.append(i)
    else:
        break

In [13]:
def final_title(raw):
    if raw in title_list:
        return raw
    else:
        return 'Other 2'
full_df['title']=full_df.title.apply(final_title)

In [14]:
def get_emp_length(raw):
    raw=str(raw)
    if raw=='n/a':
        return np.nan
    elif '+' in (raw) :
        return 10
    elif '<' in (raw) :
        return 1
    else:
        return (re.sub("\D","",raw))
def get_last_week_pay(raw):
    try:
        return (re.sub("\D","",raw))
    except:
        return -9999
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
def clean_text(raw_text):
    cleantext=np.nan
    if type(raw_text)==str:
        cleanr=re.compile('<.*?>')
        cleantext=re.sub(cleanr,' ',raw_text)
        cleantext=cleantext.replace('>','')
        cleantext=' '.join(cleantext.split())
        stop_words=set(stopwords.words("english"))
        words=word_tokenize(cleantext)
        filtered_sentence=[]
        for w in words:
            if w not in stop_words:
                filtered_sentence.append(w)
        return len(filtered_sentence)
    else:
        return 0
def verification_status_joint(raw):
    if type(raw)==str:
        return 1
    else:
        return -9999

In [20]:
full_df['term']=full_df['term'].apply(lambda x: (re.sub("\D","",str(x))))
full_df['emp_length']=full_df['emp_length'].apply(get_emp_length)
full_df['last_week_pay']=full_df['last_week_pay'].apply(get_last_week_pay)
full_df['desc']=full_df['desc'].apply(clean_text)
full_df['verification_status_joint']=full_df['verification_status_joint'].apply(verification_status_joint)

In [23]:
cols_list=full_df.columns
cols_obj=[]
obj_list=full_df.dtypes.values
for col,obj in zip(cols_list,obj_list):
    if obj==object:
        cols_obj.append(col)

In [24]:
le=preprocessing.LabelEncoder()
for col in cols_obj:
    full_df[col]=le.fit_transform(full_df[col])

In [25]:
def fill_nulls(value):
    cols_fill=['emp_length','annual_inc','delinq_2yrs','inq_last_6mths','mths_since_last_delinq','open_acc','pub_rec','revol_util','total_acc','collections_12_mths_ex_med','acc_now_delinq','tot_coll_amt','tot_cur_bal','total_rev_hi_lim']
    if value==-9999:
        for col in cols_fill:
            full_df.loc[full_df[col].isnull(),col]=-9999
    else:
        for col in cols_fill:
            full_df.loc[full_df[col].isnull(),col]=full_df[col].median()

In [26]:
fill_nulls(-9999)

In [27]:
value=-9999
def helping_features(value):
    full_df['Interest']=full_df['funded_amnt']*full_df['int_rate']*full_df['term']/(100*12)
    if value==-9999:
        full_df['Monthly_income']=full_df['annual_inc'].apply(lambda x: x/12 if x>=0 else -9999)
        full_df['Monthly_debt_repay']=full_df['Monthly_income']*full_df['dti']/100
        full_df['Monthly_debt_repay']=full_df['Monthly_debt_repay'].apply(lambda x: x if x>0 else -9999)
    else:
        full_df['Monthly_income']=full_df['annual_inc']/12
        full_df['Monthly_debt_repay']=full_df['Monthly_income']*full_df['dti']/100

In [28]:
helping_features(-9999)

In [29]:
full_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,application_type,verification_status_joint,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,nulls,Interest,Monthly_income,Monthly_debt_repay
0,14350,14350,14350.0,0,19.19,4,22,10,4,28700.0,1,0,0,2,718,334,9,33.88,0.0,1.0,50.0,14.0,1.0,22515.0,73.1,28.0,0,1173.84,0.0,0.0,0.0,0.0,0,-9999,60,0.0,0.0,28699.0,30800.0,2,0.0,2391.666667,810.296667
1,4800,4800,4800.0,0,10.99,1,8,1,1,65000.0,1,0,0,4,1311,196,20,3.64,0.0,1.0,-9999.0,6.0,0.0,7624.0,23.2,13.0,1,83.95,0.0,0.0,0.0,0.0,0,-9999,93,0.0,0.0,9974.0,32900.0,5,0.0,5416.666667,197.166667
2,10000,10000,10000.0,0,7.26,0,3,3,4,45000.0,0,0,0,2,718,430,35,18.42,0.0,0.0,-9999.0,5.0,0.0,10877.0,31.2,19.0,1,56.47,0.0,0.0,0.0,0.0,0,-9999,93,0.0,65.0,38295.0,34900.0,5,0.0,3750.000000,690.750000
3,15000,15000,15000.0,0,19.72,3,19,2,5,105000.0,0,0,0,2,718,207,45,14.97,0.0,2.0,46.0,10.0,0.0,13712.0,55.5,21.0,0,4858.62,0.0,0.0,0.0,0.0,0,-9999,15,0.0,0.0,55564.0,24700.0,3,0.0,8750.000000,1309.875000
4,16000,16000,16000.0,0,10.64,1,6,2,5,52000.0,2,0,0,1,2303,838,4,20.16,0.0,0.0,-9999.0,11.0,0.0,35835.0,76.2,27.0,1,2296.41,0.0,0.0,0.0,0.0,0,-9999,97,0.0,0.0,47159.0,47033.0,5,0.0,4333.333333,873.600000


In [30]:
full_df['unpaid_perc']=(full_df['Interest']-full_df['total_rec_int'])/full_df['Interest']

In [31]:
full_df['last_to_term']=full_df['last_week_pay']/(full_df['term']*52/12)

In [32]:
full_df['monthly_int_to_debt_repay']=full_df['Interest']/full_df['Monthly_debt_repay']/12

In [33]:
full_df['iti']=full_df['Interest']/full_df['Monthly_income']/12

In [34]:
full_df['dti_and_iti']=full_df['iti']+full_df['dti']

In [35]:
full_df['itc']=full_df['Interest']/full_df['total_rev_hi_lim']/(144)

In [36]:
cols=[]
for col in full_df.columns:
    if col not in ['Interest','total_rec_int','Monthly_income','Monthly_debt_repay','last_to_term','dti_and_iti']:
        cols.append(col)

In [43]:
train_new=full_df[:size_train]
test_new=full_df[size_train:]
train_new=train_new[cols]
test_new=test_new[cols]

In [38]:
import xgboost as xgb

In [40]:
param={}

In [44]:
param['objective']='binary:logistic'
param['eta']=0.1
param['max_depth']=10
param['silent']=0
param['eval_metric']="auc"
param['subsample']=1
param['colsample_bytree']=1
param['seed']=3
num_rounds=5000

In [ ]:
tes=test_new.drop

In [69]:
kf=KFold(n_splits=2, shuffle=True,random_state=3)
for train_index, test_index in kf.split(train_new):
    print("Split")
    d_tr=xgb.DMatrix(train_new.iloc[train_index].values, target[train_index])
    d_te=xgb.DMatrix(train_new.iloc[test_index].values, target[test_index])
    wl=[(d_tr,'train'), (d_te,'test')]
    model=xgb.train(param,d_tr,num_rounds,wl,maximize=True)
 

Exception ignored in: <bound method Booster.__del__ of <xgboost.core.Booster object at 0x000002B0A735F828>>
Traceback (most recent call last):
  File "C:\Users\pragy\Anaconda3\lib\site-packages\xgboost\core.py", line 742, in __del__
    if self.handle is not None:
AttributeError: 'Booster' object has no attribute 'handle'
Exception ignored in: <bound method Booster.__del__ of <xgboost.core.Booster object at 0x000002B099BC3828>>
Traceback (most recent call last):
  File "C:\Users\pragy\Anaconda3\lib\site-packages\xgboost\core.py", line 742, in __del__
    if self.handle is not None:
AttributeError: 'Booster' object has no attribute 'handle'
Exception ignored in: <bound method Booster.__del__ of <xgboost.core.Booster object at 0x000002B099BB8080>>
Traceback (most recent call last):
  File "C:\Users\pragy\Anaconda3\lib\site-packages\xgboost\core.py", line 742, in __del__
    if self.handle is not None:
AttributeError: 'Booster' object has no attribute 'handle'


Split
[0]	train-auc:0.834549	test-auc:0.825862
[1]	train-auc:0.837958	test-auc:0.828267
[2]	train-auc:0.837515	test-auc:0.828129
[3]	train-auc:0.837112	test-auc:0.827431
[4]	train-auc:0.835982	test-auc:0.826559
[5]	train-auc:0.835809	test-auc:0.826467
[6]	train-auc:0.835809	test-auc:0.826467
[7]	train-auc:0.835806	test-auc:0.82647
[8]	train-auc:0.835806	test-auc:0.82647
[9]	train-auc:0.835806	test-auc:0.82647
[10]	train-auc:0.838542	test-auc:0.828886
[11]	train-auc:0.838521	test-auc:0.828855
[12]	train-auc:0.83851	test-auc:0.828838
[13]	train-auc:0.83851	test-auc:0.828838
[14]	train-auc:0.83851	test-auc:0.828838
[15]	train-auc:0.83851	test-auc:0.828838
[16]	train-auc:0.83851	test-auc:0.828838
[17]	train-auc:0.83851	test-auc:0.828838
[18]	train-auc:0.83851	test-auc:0.828838
[19]	train-auc:0.83851	test-auc:0.828838
[20]	train-auc:0.83851	test-auc:0.828838
[21]	train-auc:0.83851	test-auc:0.828838
[22]	train-auc:0.83851	test-auc:0.828838
[23]	train-auc:0.83851	test-auc:0.828838
[24]	train-

[198]	train-auc:0.83851	test-auc:0.828838
[199]	train-auc:0.83851	test-auc:0.828838
[200]	train-auc:0.83851	test-auc:0.828838
[201]	train-auc:0.83851	test-auc:0.828838
[202]	train-auc:0.83851	test-auc:0.828838
[203]	train-auc:0.83851	test-auc:0.828838
[204]	train-auc:0.83851	test-auc:0.828838
[205]	train-auc:0.83851	test-auc:0.828838
[206]	train-auc:0.83851	test-auc:0.828838
[207]	train-auc:0.83851	test-auc:0.828838
[208]	train-auc:0.83851	test-auc:0.828838
[209]	train-auc:0.83851	test-auc:0.828838
[210]	train-auc:0.83851	test-auc:0.828838
[211]	train-auc:0.83851	test-auc:0.828838
[212]	train-auc:0.83851	test-auc:0.828838
[213]	train-auc:0.83851	test-auc:0.828838
[214]	train-auc:0.83851	test-auc:0.828838
[215]	train-auc:0.83851	test-auc:0.828838
[216]	train-auc:0.83851	test-auc:0.828838
[217]	train-auc:0.83851	test-auc:0.828838
[218]	train-auc:0.83851	test-auc:0.828838
[219]	train-auc:0.83851	test-auc:0.828838
[220]	train-auc:0.83851	test-auc:0.828838
[221]	train-auc:0.83851	test-auc:0

[394]	train-auc:0.83851	test-auc:0.828838
[395]	train-auc:0.83851	test-auc:0.828838
[396]	train-auc:0.83851	test-auc:0.828838
[397]	train-auc:0.83851	test-auc:0.828838
[398]	train-auc:0.83851	test-auc:0.828838
[399]	train-auc:0.83851	test-auc:0.828838
[400]	train-auc:0.83851	test-auc:0.828838
[401]	train-auc:0.83851	test-auc:0.828838
[402]	train-auc:0.83851	test-auc:0.828838
[403]	train-auc:0.83851	test-auc:0.828838
[404]	train-auc:0.83851	test-auc:0.828838
[405]	train-auc:0.83851	test-auc:0.828838
[406]	train-auc:0.83851	test-auc:0.828838
[407]	train-auc:0.83851	test-auc:0.828838
[408]	train-auc:0.83851	test-auc:0.828838
[409]	train-auc:0.83851	test-auc:0.828838
[410]	train-auc:0.83851	test-auc:0.828838
[411]	train-auc:0.83851	test-auc:0.828838
[412]	train-auc:0.83851	test-auc:0.828838
[413]	train-auc:0.83851	test-auc:0.828838
[414]	train-auc:0.83851	test-auc:0.828838
[415]	train-auc:0.83851	test-auc:0.828838
[416]	train-auc:0.83851	test-auc:0.828838
[417]	train-auc:0.83851	test-auc:0

[590]	train-auc:0.83851	test-auc:0.828838
[591]	train-auc:0.83851	test-auc:0.828838
[592]	train-auc:0.83851	test-auc:0.828838
[593]	train-auc:0.83851	test-auc:0.828838
[594]	train-auc:0.83851	test-auc:0.828838
[595]	train-auc:0.83851	test-auc:0.828838
[596]	train-auc:0.83851	test-auc:0.828838
[597]	train-auc:0.83851	test-auc:0.828838
[598]	train-auc:0.83851	test-auc:0.828838
[599]	train-auc:0.83851	test-auc:0.828838
[600]	train-auc:0.83851	test-auc:0.828838
[601]	train-auc:0.83851	test-auc:0.828838
[602]	train-auc:0.83851	test-auc:0.828838
[603]	train-auc:0.83851	test-auc:0.828838
[604]	train-auc:0.83851	test-auc:0.828838
[605]	train-auc:0.83851	test-auc:0.828838
[606]	train-auc:0.83851	test-auc:0.828838
[607]	train-auc:0.83851	test-auc:0.828838
[608]	train-auc:0.83851	test-auc:0.828838
[609]	train-auc:0.83851	test-auc:0.828838
[610]	train-auc:0.83851	test-auc:0.828838
[611]	train-auc:0.83851	test-auc:0.828838
[612]	train-auc:0.83851	test-auc:0.828838
[613]	train-auc:0.83851	test-auc:0

...

[3914]	train-auc:1	test-auc:0.948932
[3915]	train-auc:1	test-auc:0.948931
[3916]	train-auc:1	test-auc:0.948933
[3917]	train-auc:1	test-auc:0.948933
[3918]	train-auc:1	test-auc:0.948933
[3919]	train-auc:1	test-auc:0.948932
[3920]	train-auc:1	test-auc:0.94893
[3921]	train-auc:1	test-auc:0.948929
[3922]	train-auc:1	test-auc:0.948931
[3923]	train-auc:1	test-auc:0.948932
[3924]	train-auc:1	test-auc:0.948931
[3925]	train-auc:1	test-auc:0.948931
[3926]	train-auc:1	test-auc:0.948928
[3927]	train-auc:1	test-auc:0.948926
[3928]	train-auc:1	test-auc:0.948927
[3929]	train-auc:1	test-auc:0.948926
[3930]	train-auc:1	test-auc:0.948926
[3931]	train-auc:1	test-auc:0.948927
[3932]	train-auc:1	test-auc:0.94893
[3933]	train-auc:1	test-auc:0.948929
[3934]	train-auc:1	test-auc:0.948929
[3935]	train-auc:1	test-auc:0.948931
[3936]	train-auc:1	test-auc:0.948931
[3937]	train-auc:1	test-auc:0.948936
[3938]	train-auc:1	test-auc:0.948938
[3939]	train-auc:1	test-auc:0.948939
[3940]	train-auc:1	test-auc:0.948938
[39

[4136]	train-auc:1	test-auc:0.948837
[4137]	train-auc:1	test-auc:0.948837
[4138]	train-auc:1	test-auc:0.948839
[4139]	train-auc:1	test-auc:0.94884
[4140]	train-auc:1	test-auc:0.948839
[4141]	train-auc:1	test-auc:0.94884
[4142]	train-auc:1	test-auc:0.948841
[4143]	train-auc:1	test-auc:0.948838
[4144]	train-auc:1	test-auc:0.948838
[4145]	train-auc:1	test-auc:0.948837
[4146]	train-auc:1	test-auc:0.948836
[4147]	train-auc:1	test-auc:0.948832
[4148]	train-auc:1	test-auc:0.948832
[4149]	train-auc:1	test-auc:0.948828
[4150]	train-auc:1	test-auc:0.94883
[4151]	train-auc:1	test-auc:0.948828
[4152]	train-auc:1	test-auc:0.948826
[4153]	train-auc:1	test-auc:0.948824
[4154]	train-auc:1	test-auc:0.948824
[4155]	train-auc:1	test-auc:0.948822
[4156]	train-auc:1	test-auc:0.948821
[4157]	train-auc:1	test-auc:0.94882
[4158]	train-auc:1	test-auc:0.948819
[4159]	train-auc:1	test-auc:0.948818
[4160]	train-auc:1	test-auc:0.94882
[4161]	train-auc:1	test-auc:0.948814
[4162]	train-auc:1	test-auc:0.948814
[4163]

[4358]	train-auc:1	test-auc:0.948763
[4359]	train-auc:1	test-auc:0.948764
[4360]	train-auc:1	test-auc:0.948765
[4361]	train-auc:1	test-auc:0.948767
[4362]	train-auc:1	test-auc:0.948766
[4363]	train-auc:1	test-auc:0.948764
[4364]	train-auc:1	test-auc:0.948763
[4365]	train-auc:1	test-auc:0.948762
[4366]	train-auc:1	test-auc:0.948762
[4367]	train-auc:1	test-auc:0.948764
[4368]	train-auc:1	test-auc:0.948762
[4369]	train-auc:1	test-auc:0.948761
[4370]	train-auc:1	test-auc:0.948762
[4371]	train-auc:1	test-auc:0.948761
[4372]	train-auc:1	test-auc:0.948759
[4373]	train-auc:1	test-auc:0.94876
[4374]	train-auc:1	test-auc:0.948759
[4375]	train-auc:1	test-auc:0.94876
[4376]	train-auc:1	test-auc:0.948759
[4377]	train-auc:1	test-auc:0.948758
[4378]	train-auc:1	test-auc:0.948756
[4379]	train-auc:1	test-auc:0.948757
[4380]	train-auc:1	test-auc:0.948756
[4381]	train-auc:1	test-auc:0.948759
[4382]	train-auc:1	test-auc:0.948757
[4383]	train-auc:1	test-auc:0.948753
[4384]	train-auc:1	test-auc:0.948754
[43

[4581]	train-auc:1	test-auc:0.948724
[4582]	train-auc:1	test-auc:0.948724
[4583]	train-auc:1	test-auc:0.948724
[4584]	train-auc:1	test-auc:0.948727
[4585]	train-auc:1	test-auc:0.948725
[4586]	train-auc:1	test-auc:0.948725
[4587]	train-auc:1	test-auc:0.948724
[4588]	train-auc:1	test-auc:0.948724
[4589]	train-auc:1	test-auc:0.948725
[4590]	train-auc:1	test-auc:0.948726
[4591]	train-auc:1	test-auc:0.948726
[4592]	train-auc:1	test-auc:0.948726
[4593]	train-auc:1	test-auc:0.948727
[4594]	train-auc:1	test-auc:0.948724
[4595]	train-auc:1	test-auc:0.948722
[4596]	train-auc:1	test-auc:0.948722
[4597]	train-auc:1	test-auc:0.948722
[4598]	train-auc:1	test-auc:0.948721
[4599]	train-auc:1	test-auc:0.94872
[4600]	train-auc:1	test-auc:0.948722
[4601]	train-auc:1	test-auc:0.948721
[4602]	train-auc:1	test-auc:0.948722
[4603]	train-auc:1	test-auc:0.948719
[4604]	train-auc:1	test-auc:0.948717
[4605]	train-auc:1	test-auc:0.948717
[4606]	train-auc:1	test-auc:0.948717
[4607]	train-auc:1	test-auc:0.948719
[4

[4804]	train-auc:1	test-auc:0.94864
[4805]	train-auc:1	test-auc:0.948639
[4806]	train-auc:1	test-auc:0.94864
[4807]	train-auc:1	test-auc:0.948638
[4808]	train-auc:1	test-auc:0.948638
[4809]	train-auc:1	test-auc:0.948637
[4810]	train-auc:1	test-auc:0.948637
[4811]	train-auc:1	test-auc:0.948637
[4812]	train-auc:1	test-auc:0.948636
[4813]	train-auc:1	test-auc:0.948635
[4814]	train-auc:1	test-auc:0.948634
[4815]	train-auc:1	test-auc:0.948636
[4816]	train-auc:1	test-auc:0.948637
[4817]	train-auc:1	test-auc:0.948638
[4818]	train-auc:1	test-auc:0.948636
[4819]	train-auc:1	test-auc:0.948636
[4820]	train-auc:1	test-auc:0.948637
[4821]	train-auc:1	test-auc:0.948634
[4822]	train-auc:1	test-auc:0.948635
[4823]	train-auc:1	test-auc:0.948636
[4824]	train-auc:1	test-auc:0.948635
[4825]	train-auc:1	test-auc:0.948633
[4826]	train-auc:1	test-auc:0.948633
[4827]	train-auc:1	test-auc:0.948632
[4828]	train-auc:1	test-auc:0.948633
[4829]	train-auc:1	test-auc:0.948633
[4830]	train-auc:1	test-auc:0.948631
[48

In [102]:
for train_index, test_index in kf.split(train_new):
    pred=model.predict(d_te)
    preds.append(list(pred))

In [110]:
(preds[0][1])

0.00019848565

In [ ]:
early stopping

In [99]:
test_new.drop('loan_status')

ValueError: labels ['loan_status'] not contained in axis

In [92]:
preds=[]

In [ ]:
xp=[]
for i in range(len())

In [89]:
preds[1]

[0.0025932689,
 0.00019848565,
 0.04533935,
 0.1408728,
 0.25301757,
 0.0029910181,
 0.029324153,
 0.13299784,
 0.01892083,
 0.0005994464,
 5.7609213e-06,
 0.011202339,
 0.021413127,
 0.0003713936,
 0.04481009,
 0.011046944,
 0.00062409695,
 0.12649956,
 3.6165568e-06,
 1.8031707e-05,
 0.02147515,
 0.00039296376,
 1.6139238e-07,
 0.0017080406,
 0.9995239,
 0.99261415,
 1.1799359e-06,
 0.9364142,
 0.2720293,
 5.0473653e-05,
 2.9700193e-06,
 9.385156e-07,
 0.8079369,
 0.00035745147,
 0.9990803,
 0.0001671243,
 0.9999474,
 0.020228283,
 0.083123505,
 0.999982,
 6.1967254e-10,
 0.6524577,
 0.00021627045,
 7.2184375e-05,
 0.0009226605,
 0.006158415,
 1.8147439e-06,
 2.7048478e-05,
 0.00032861307,
 0.0003397922,
 0.008244301,
 0.00028245331,
 0.02639406,
 0.024013104,
 7.854171e-05,
 1.2121659e-09,
 0.01046756,
 1.2557486e-05,
 6.4896645e-05,
 8.389758e-05,
 0.0017763197,
 0.9998659,
 0.5353857,
 2.829605e-06,
 0.0006176423,
 0.00023688945,
 0.9999968,
 0.001340129,
 0.0007937242,
 2.326864e

In [78]:
sb=pd.DataFrame()

In [56]:
from sklearn.model_selection import cross_val_score

In [54]:
from xgboost import XGBClassifier